In [23]:
import random

import pandas as pd
import tkinter as tk
import numpy as np
from tkinter import *
from tkinter import filedialog
from tkinter.filedialog import askopenfile
import matplotlib.pyplot as plt
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
from matplotlib.figure import Figure
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
import warnings
from sklearn.datasets import load_iris
warnings.filterwarnings('ignore')
from matplotlib.colors import ListedColormap
import seaborn as sns


class Backpropagation:
    def __init__(self,  num_of_layers, layers, learning_rate, n_iters, bias, AC_function, X_train, y_train):
        self.bias = bias
        self.eta = learning_rate
        self.m = n_iters
        self.layers = layers
        self.AC_function=AC_function
        self.Weights = []
        self.f_net=[]
        self.gradient = []
        self.X_train=X_train
        self.y_train=y_train

        for i in np.arange(0, len(layers) - 2):
            # randomly initialize a weight matrix connecting the
            # number of nodes in each respective layer together,
            # adding an extra node for the bias
            w = np.random.rand(self.layers[i] + self.bias, self.layers[i + 1] + self.bias)
            self.Weights.append(w)
        w = np.random.rand(self.layers[-2] + self.bias, self.layers[-1])
        self.Weights.append(w)

    def Activation_func(self, x,function):
        if function=="Sigmoid":
            return 1 / (1 + np.exp(-x))
        else:
            return (np.exp(x) - np.exp(-x)) / (np.exp(x) + np.exp(-x))

    def accuracy(self, y_pred, y_true):
        acc = y_pred.argmax(axis=1) == y_true.argmax(axis=1)
        return acc.mean()

    def train(self):
        for itr in range(self.m):
            self.f_net.clear()
            self.gradient.clear()
            self.feed_forward(self.X_train)
            self.backprobagate()
            self.update()

    def feed_forward(self, x):
        Z1 = np.dot(x, self.Weights[0])
        A1 = self.Activation_func(Z1, self.AC_function)
        self.f_net.append(A1)
        count = 0

        for i in np.arange(1, len(self.Weights) - 1):
            z = np.dot(self.f_net[count], self.Weights[i])
            A1 = self.Activation_func(z, self.AC_function)
            self.f_net.append(A1)
            count += 1
        z = np.dot(self.f_net[-1], self.Weights[-1])
        A1 = self.Activation_func(z, self.AC_function)
        self.f_net.append(A1)

    def backprobagate(self):
        y_pred = self.f_net[-1]
        E1 = y_pred - self.y_train
        if self.AC_function == "Sigmoid":
            dw1 = E1 * y_pred * (1 - y_pred)
        else:
            dw1 = E1 * (1 - np.power((np.exp(y_pred) - np.exp(-y_pred)) / (np.exp(y_pred) + np.exp(-y_pred)), 2))
        self.gradient.append(dw1)
        dw2 = dw1

        for i in reversed(range(len(self.f_net) - 1)):
            E2 = np.dot(dw2, self.Weights[i+1].T)
            if self.AC_function == "Sigmoid":
                dw2 = E2 * self.f_net[i] * (1 - self.f_net[i])
            else:
                dw2 = E2 * (1 - np.power((np.exp(self.f_net[i]) - np.exp(-self.f_net[i])) / (np.exp(self.f_net[i]) + np.exp(-self.f_net[i])), 2))
            self.gradient.insert(0, dw2)

    def update(self):

        self.Weights[0] = self.Weights[0] - self.eta * np.dot(self.X_train.T, self.gradient[0])
        for i in np.arange(1, len(self.Weights)):
            self.Weights[i] = self.Weights[i] - self.eta * np.dot(self.f_net[i-1].T, self.gradient[i])


def upload_file():
    file = filedialog.askopenfilename()
    if file:  # user selected one file
        fob = open(file, 'r')

        global loaded_data  # data loaded from file
        loaded_data = pd.read_csv(file)
        print(loaded_data)

        pre_process()

        str1 = "Rows:" + str(loaded_data.shape[0]) + "\nColumns:" + str(loaded_data.shape[1])
        display_box.insert(tk.END, str1)  # add to Text widget
    else:  # user cancel the file browser window
        display_box.insert(tk.END, 'No file chosen')

def selected_item():
    if len(AC_function_listbox.curselection()) == 1:
        global activation_function
        for i in AC_function_listbox.curselection():
            activation_function = AC_function_listbox.get(i)
    else:
        tk.messagebox.showerror('Selection error', 'you have to select one activation function')

def confirm_neurons():
    global hidden_neurons_num
    temp = HLN_var.get().split(',')
    hidden_neurons_num = [int(x) for x in temp]
    hidden_neurons_num.insert(0, 5)
    hidden_neurons_num.append(3)
    if len(hidden_neurons_num) < int(HL_var.get()):
        tk.messagebox.showerror('neurons number error', 'you have to type the number of neurons like: 10,11,12')

def pre_process():
    data_scaled = loaded_data.copy()
    normalizing_columns = ['bill length', 'bill depth', 'flipper length', 'body mass']
    for i in normalizing_columns:
        loaded_data[i] = (data_scaled[i] - data_scaled[i].min()) / (data_scaled[i].max() - data_scaled[i].min())

    loaded_data['gender'].fillna('male', inplace=True)  # fill the nulls
    loaded_data['gender'] = preprocessing.LabelEncoder().fit_transform(loaded_data['gender'])
    loaded_data['species'] = preprocessing.LabelEncoder().fit_transform(loaded_data['species'])
import seaborn as sns
def confusion_matrix(y_test, predictions): 
    confusionMatrix=np.zeros(len(y_test),len(y_test))
    #          Predicted condition
    #True positive (TP)   False negative (FN)
    #False positive (FP)  True negative (TN)
    for i in range(len(y_test)):
        #1=positive, 0=negative
        if predictions[i] ==y_test[i]: 
            confusionMatrix[i,i]+=1 #True Positives
            
        elif predictions[i] > y_test[i]:
            confusionMatrix[i,i+1]+=1 #False Positives
            
        elif predictions[i] < y_test[i]:
            confusionMatrix[i+1,i]+=1 #False Negatives
            
     
    
    
    fig = plt.figure(facecolor='#ffffff' , edgecolor='k' , figsize=(5 , 3))
    ax = sns.heatmap(confusionMatrix, annot=True, cmap='Blues')
    ax.set_xlabel('\nPredicted Values')
    ax.set_ylabel('Actual Values ');

    ## Ticket labels - List must be in alphabetical order
    ax.xaxis.set_ticklabels([classes[1],classes[0]])
    ax.yaxis.set_ticklabels([classes[1],classes[0]])
    ## Display the visualization of the Confusion Matrix.
    plt.show()
    canvas = FigureCanvasTkAgg(fig, master = window)
    canvas.get_tk_widget().grid(row = 3, column = 2,rowspan = 200, sticky=tk.E,padx=15)
    canvas.draw()
    window.grid_rowconfigure(6, weight=0)

window = tk.Tk()
window.geometry("900x600")  # Size of the window
window.title('Penguins classification')  # window title

window_font = ('times', 12, 'bold')  #

l1 = tk.Label(window, text='Read File & create DataFrame', width=30, font=window_font).grid(row=1, column=1)

load_data_button = tk.Button(window, text='Load Data', width=20, command=lambda: upload_file()).grid(row=2, column=1)

display_box = tk.Text(window, width=40, height=5)
display_box.grid(row=3, column=1, padx=5)
bias_check = IntVar()
check_bias_button = Checkbutton(window, text="Add bias", variable=bias_check).grid(row=4, column=1)

m_var = tk.StringVar()
eta_var = tk.StringVar()
HL_var = tk.StringVar()
HLN_var = tk.StringVar()

# taking the learning rate
l2 = tk.Label(window, text='Enter learning rate').grid(row=5, column=1)
eta = tk.Entry(window, textvariable=eta_var).grid(row=6, column=1)

# taking number of epochs
l3 = tk.Label(window, text='Enter number of epochs').grid(row=7, column=1)
m = tk.Entry(window, textvariable=m_var).grid(row=8, column=1)

# taking number of hidden layers
l5 = tk.Label(window, text='Enter number of hidding layers').grid(row=9, column=1)
hiddin_layers = tk.Entry(window, textvariable=HL_var).grid(row=10, column=1)

l3 = tk.Label(window, text='number of neurons').grid(row=12, column=1, sticky=tk.E, padx=12)
hidden_layers_nuerons = tk.Entry(window, textvariable=HLN_var).grid(row=14, column=1, sticky=tk.E, ipady=15)

l4 = tk.Label(window, text='Select one activation function').grid(row=12, column=1, sticky=tk.W)

# adding and selecting features to use
AC_function_listbox = tk.Listbox(window, selectmode=tk.MULTIPLE, exportselection=False, height=3)
AC_function_listbox.grid(row=14, column=1, sticky=tk.W)
x = ["Sigmoid", "Tanh", ]
for each_item in range(len(x)):
    AC_function_listbox.insert(each_item, x[each_item])


def run():
    # see if bias is checked to replace it with 1
    bias = 0
    if (bias_check.get() == 1):
        bias = 1
    X = loaded_data.iloc[:, 1:6]
    X = X.to_numpy()
    Y = loaded_data.iloc[:, 0]
    Y = pd.get_dummies(Y).values
    X_train, X_test, y_train, y_test = train_test_split(X, Y, random_state=104, test_size=0.4, shuffle=True)
    if bias == 1:
        X_test = np.c_[X_test, np.ones((X_test.shape[0]))]
        X_train = np.c_[ X_train, np.ones(( X_train.shape[0]))]
        
    B1 = Backpropagation(float(HL_var.get()), hidden_neurons_num, float(eta_var.get()), int(m_var.get()), bias, activation_function, X_train, y_train)
    B1.train()
    B1.f_net.clear()
   
    B1.feed_forward(X_test)
    Testing_accuracy = B1.accuracy(B1.f_net[-1], y_test)
    str2="Testing accuracy: " + str(Testing_accuracy)
    
    B1.f_net.clear()
   
    B1.feed_forward(X_train)
    Training_accuracy = B1.accuracy(B1.f_net[-1], y_train)
    str3="Training accuracy: " + str(Training_accuracy)
    
  
    print(str2)
    print(str3)
    
    
    #confusionMAt = tk.Button(window, text='Confution Matrix',width=20,command = lambda:confusion_matrix(y_test, B1.f_net[-1])).grid(row=18,column=1, sticky=tk.W)      
    display_box1.delete("1.0","end")
    display_box1.insert(tk.END, str2 )
    
    

select = tk.Button(window, text='Select', width=16, command=lambda: selected_item()).grid(row=15, column=1, sticky= tk.W)
confirm = tk.Button(window, text='Confirm', width=16, command=lambda: confirm_neurons()).grid(row=15, column=1, sticky= tk.E)

run_button = tk.Button(window, text='Run', width=20, command=lambda: run()).grid(row=16, column=1)
display_box1 = tk.Text(window, width=40, height=5)
display_box1.grid(row=17, column=1, padx=5)

window.mainloop()

       species  bill length  bill depth  flipper length  gender  body mass
0       Adelie         39.1        18.7             181    male       3750
1       Adelie         39.5        17.4             186  female       3800
2       Adelie         40.3        18.0             195  female       3250
3       Adelie         39.6        17.7             186  female       3500
4       Adelie         36.7        19.3             193  female       3450
..         ...          ...         ...             ...     ...        ...
145  Chinstrap         50.8        18.5             201    male       4450
146  Chinstrap         50.1        17.9             190  female       3400
147  Chinstrap         49.0        19.6             212    male       4300
148  Chinstrap         51.5        18.7             187    male       3250
149  Chinstrap         49.8        17.3             198  female       3675

[150 rows x 6 columns]
Testing accuracy: 0.25
Training accuracy: 0.3888888888888889
Testing accurac